In [1]:
import altair as alt
import pandas as pd
from pony.orm import *
from datetime import datetime
import random
from bail.db import DB, Case, Inmate

In [2]:
# Connect to SQLite database using Pony
db = DB()

In [29]:
statuses = set(select(i.status for i in Inmate))
pretrial = set(s for s in statuses if ("Prearraignment" in s or "Pretrial" in s))
  
pretrial_inmates = list(select(i for i in Inmate if i.status in pretrial))
other_inmates = list(select(i for i in Inmate if not i.status in pretrial))

all_inmates = list(list(select(i for i in Inmate)))
psb_inmates = list(select(i for i in Inmate if i.building == "PUBLIC SAFETY BUILDING"))
psb_pre_inmates = list(select(i for i in Inmate if i.building == "PUBLIC SAFETY BUILDING" and i.status in pretrial))
ccb_inmates = list(select(i for i in Inmate if i.building == "CITY COUNTY BUILDING"))
ccb_pre_inmates = list(select(i for i in Inmate if i.building == "CITY COUNTY BUILDING" and i.status in pretrial))
ccb_intransit_inmates = list(select(i for i in Inmate if i.building == "CITY COUNTY BUILDING" and "Intransit" in i.status))

print(f"Pretrial: {len(pretrial_inmates)} Other: {len(other_inmates)} All: {len(all_inmates)}")
print(f"All CCB: {len(ccb_inmates)} Pre: {len(ccb_pre_inmates)} Intransit: {len(ccb_intransit_inmates)}")

print(f"Ratio CCB pre-trial: {len(ccb_pre_inmates)/len(ccb_inmates)}")
print(f"Ratio non-CCB pre-trial: {len(psb_pre_inmates)/len(psb_inmates)}")

#for i in all_inmates:
#    print(i.building, i.status)

Pretrial: 340 Other: 434 All: 774
All CCB: 263 Pre: 157 Intransit: 26
Ratio CCB pre-trial: 0.596958174904943
Ratio non-CCB pre-trial: 0.46134020618556704


In [24]:
157/263, 340/len(all_inmates)

(0.596958174904943, 0.4392764857881137)

In [3]:

pretrial_inmates = list(select(i for i in Inmate if i.status in pretrial))
other_inmates = list(select(i for i in Inmate if not i.status in pretrial))
  
black_pretrial_inmates = list(select(i for i in Inmate if i.race == "African American" and i.status in pretrial))
white_pretrial_inmates = list(select(i for i in Inmate if i.race == "Caucasian" and i.status in pretrial))
other_pretrial_inmates = list(select(i for i in Inmate if 
                                   i.race != "African American" and 
                                   i.race != "Caucasian" and
                                   i.status in pretrial))

In [4]:
def severity(inmate):
    if any("Felony" in x.severity for c in inmate.cases for x in c.charges):
        return 'felony'
    else:
        return 'misdemeanor'
    
def bail_value(inmate):
    bails = [c.cash_bond for c in inmate.cases if c.cash_bond]
    if len(bails) == 0:
        bails = [0]
    return max(bails)

def build_data(inmates):
    return pd.DataFrame([{
        'race': x.race,
        'bail': bail_value(x),
        'days_since_booking': x.days_since_booking(),
        'most_recent_arrest': x.most_recent_arrest(),
        'days_since_most_recent_arrest': x.days_since_most_recent_arrest(),
        'severity': severity(x),
        'url': x.url,
    } for x in inmates])

data = build_data(pretrial_inmates)

In [6]:
days = data.query('severity=="felony"')['days_since_booking']
print(f"Mean days in jail for a felony: {days.mean()} with standard deviation {days.std()} and maximum {days.max()}")

days = data.query('severity=="misdemeanor"')['days_since_booking']
print(f"Mean days in jail for a misdemeanor: {days.mean()} with standard deviation {days.std()} and maximum {days.max()}")

Mean days in jail for a felony: 180.38636363636363 with standard deviation 193.9453196470258 and maximum 1237
Mean days in jail for a misdemeanor: 74.70149253731343 with standard deviation 120.43469435257926 and maximum 603


In [7]:
# Instead try most recent arrest date...
days = data.query('severity=="felony"')['days_since_most_recent_arrest']
print(f"Mean days since arrest for a felony: {days.mean()} with standard deviation {days.std()} and maximum {days.max()}")

days = data.query('severity=="misdemeanor"')['days_since_most_recent_arrest']
print(f"Mean days since arrest for a misdemeanor: {days.mean()} with standard deviation {days.std()} and maximum {days.max()}")

Mean days since arrest for a felony: 148.4715909090909 with standard deviation 174.04340768825534 and maximum 1237
Mean days since arrest for a misdemeanor: 58.865671641791046 with standard deviation 81.57352057042888 and maximum 419


In [8]:
data.query('severity=="misdemeanor"').sort_values('days_since_most_recent_arrest')

,race,bail,days_since_booking,most_recent_arrest,days_since_most_recent_arrest,severity,url
237,,0.0,11,2021-04-06 23:16:00,11,misdemeanor,https://danesheriff.com/Inmates/Detail/581210
85,,0.0,11,2021-04-07 07:05:00,11,misdemeanor,https://danesheriff.com/Inmates/Detail/15544
63,,0.0,12,2021-04-06 13:06:00,12,misdemeanor,https://danesheriff.com/Inmates/Detail/212053
153,,0.0,12,2021-04-06 02:33:00,12,misdemeanor,https://danesheriff.com/Inmates/Detail/177662
119,,0.0,12,2021-04-06 15:45:00,12,misdemeanor,https://danesheriff.com/Inmates/Detail/188793
...,...,...,...,...,...,...,...
87,African American,0.0,603,2020-10-05 16:03:00,195,misdemeanor,https://danesheriff.com/Inmates/Detail/90103804
58,Hispanic,100.0,502,2020-08-10 15:42:00,251,misdemeanor,https://danesheriff.com/Inmates/Detail/426290
143,African American,0.0,334,2020-06-04 15:12:00,318,misdemeanor,https://danesheriff.com/Inmates/Detail/451638
4,Caucasian,100.0,325,2020-06-01 15:11:00,321,misdemeanor,https://danesheriff.com/Inmates/Detail/90126213


In [ ]:
# Prefab stable colors for charts
domain = ['African American', 'American Indian or Alaskan Native', 'Asian or Pacific Islander', 'Caucasian', 'Hispanic', 'Unknown']
range_ = ['red', 'green', 'blue', 'orange', 'purple', 'grey']
race = alt.Color('race:N', scale=alt.Scale(domain=domain, range=range_))

# Facet across both felonies and misdemeanors
alt.Chart(data).mark_circle(size=40, opacity=0.85, clip=True).encode(
    y=alt.Y('days_in_prison:Q',
        scale=alt.Scale(domain=(0, 365))
    ),
    x=alt.X('bail:Q',
        scale=alt.Scale(domain=(0, 1000))
    ),
    color=race,
).facet(
    row='severity:N'
)

In [ ]:
# Misdemeanors only with trend lines
misdemeanors = data.query('severity=="misdemeanor"')

base = alt.Chart(misdemeanors).mark_circle(size=30, clip=True).encode(
    y=alt.Y('days_in_prison:Q',
        scale=alt.Scale(domain=(0, 365))
    ),
    x=alt.X('bail:Q',
        scale=alt.Scale(domain=(0, 800))
    ),
    color=race,
)


polynomial_fit = [
    base.transform_regression(
        "x", "y", method="poly", order=1, as_=["x", str(thing)]
    )
    .mark_line()
    .transform_fold([str(thing)], as_=["race", "y"])
    .encode(race)
    for thing in ['African American', 'Caucasian']
]

alt.layer(base, *polynomial_fit)
base